In [22]:
# imports
import pandas as pd

# datasources
# CSV
# trees = pd.read_csv(r'/Users/war11395/Documents/Blog/NY Forestry/Data/Forestry_Tree_Points.csv')
# insps = pd.read_csv(r'/Users/war11395/Documents/Blog/NY Forestry/Data/Forestry_Inspections.csv')

# Insights
# trees = %dataset(ny_forestry_trees_551c3de)
# insps = %dataset(ny_forestry_inspecti_3d2f6bf)

# GitHub
trees = pd.read_csv("https://raw.githubusercontent.com/WarrenDz/ny-forestry-am/main/Data/Forestry_Tree_Points_subset.csv")
insps = pd.read_csv("https://raw.githubusercontent.com/WarrenDz/ny-forestry-am/main/Data/Forestry_Inspections_subset.csv")

In [25]:
# INSPECTIONS
# Drop null inspections with NULL InspectionDates
insps = insps.dropna(subset=['InspectionDate'])

# Drop columns we aren't going to use
insps = insps.drop(['Location', 'Geometry', 'UpdatedDate', 'ReinspectionDate', 'SWLocationRating', 'SWClearanceRating', 'SWDamageRating', 'SWDamagedFlagstonesRating', 'SWRatingTotal'], axis=1)

# Filter rows where the inspection date is not a valid date
insps.filter(regex=r'^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', axis=0)

,OBJECTID,InspectionType,InspectionStatus,InspectionTPCondition,InspectionTPStructure,TreePointDBH,SWTreeConditionRating,TreePointGlobalID,PlantingSpaceGlobalID,ServiceRequestGlobalID,GlobalID,InspectionDate,ClosedDate,CreatedDate,ParentInspectionGlobalID,SWTotalTrees


In [26]:
## TREES
# Drop rows missing location
trees = trees.dropna(subset=['Location'])
trees = trees.replace(regex='^202-', value='2020-')

# Strip parentheses from location column
trees['Location'] = trees['Location'].str.strip('()')

# Split location column into separate columns for mapping
trees[['Lat', 'Long']] = trees['Location'].str.split(', ', expand=True)

# Drop original location and geometry columns
trees = trees.drop(['Location', 'Geometry'], axis=1)

In [27]:
## MERGE INSPECTIONS with TREES
# Convert trees to additional df and retain columns we'd like to join with our inspections
tree_ages = trees[['GlobalID', 'PlantedDate']]

# Join trees to inspection
insps = pd.merge(left=insps, right=tree_ages, how='inner', left_on='TreePointGlobalID', right_on='GlobalID', copy=True)

# Convert fields to datetimes
insps['PlantedDate'] = pd.to_datetime(insps['PlantedDate'], errors='coerce')
insps['InspectionDate'] = pd.to_datetime(insps['InspectionDate'], errors='coerce')

# Calculate difference between date of planting and date of inspection
insps['DaysSincePlant'] = (insps['InspectionDate'] - insps['PlantedDate']).dt.days
insps = insps.fillna(0)
insps.astype({'DaysSincePlant': 'int32'}).dtypes
insps = insps.sort_values(by='DaysSincePlant', ascending=False)

,OBJECTID,InspectionType,InspectionStatus,InspectionTPCondition,InspectionTPStructure,TreePointDBH,SWTreeConditionRating,TreePointGlobalID,PlantingSpaceGlobalID,ServiceRequestGlobalID,GlobalID_x,InspectionDate,ClosedDate,CreatedDate,ParentInspectionGlobalID,SWTotalTrees,GlobalID_y,PlantedDate,DaysSincePlant
0,1587820,Level 2,Closed,Fair,Full,7.0,0.0,252D2259-FE43-48F0-897E-07C2EE3EDDB6,E0836477-06A3-42AA-B11B-87D413485701,0,22A57722-4E07-4730-A411-C088AE8A270F,2019-10-01 14:54:00,10/1/19 14:54,10/1/19 14:54,0,0.0,252D2259-FE43-48F0-897E-07C2EE3EDDB6,0,0.0
1,8812725,Level 2,Closed,Good,Full,24.0,0.0,B9FD0447-8673-4FEF-A854-A90E916A2770,1B35641A-98AD-4812-8274-2E7910308FBC,0,FD49DD81-832E-4F67-9D68-A0F515F52853,2019-07-02 14:51:00,7/2/19 14:51,7/2/19 14:51,0,0.0,B9FD0447-8673-4FEF-A854-A90E916A2770,0,0.0
18,1574835,Level 2,Closed,Good,Full,9.0,0.0,96A6F5CE-FE48-4334-B5A2-340E41BF5343,47A98A45-5358-4392-90B2-2C80EA6350EC,0,A267F9FE-5E45-435F-BE4B-D78C20E1F352,2019-09-30 20:00:00,9/30/19 20:00,9/30/19 20:00,0,0.0,96A6F5CE-FE48-4334-B5A2-340E41BF5343,0,0.0
17,1801636,Level 2,Closed,Fair,Full,21.0,0.0,E9FC3C52-8FB7-4E6B-95B1-466C20417D14,A1E5DB11-542B-4A9A-ADA9-B9AE29EDF2B0,0,2B738E50-A1AA-4FB8-9463-88CBC8886CCE,2019-10-24 04:00:00,10/29/19 18:43,10/29/19 18:42,0,0.0,E9FC3C52-8FB7-4E6B-95B1-466C20417D14,0,0.0
16,1753727,Level 2,Closed,Good,Full,8.0,0.0,5B3D4939-CBAB-45F6-BC37-6BFB0EC5DA15,CF0E2A6C-F035-4CC8-931C-A03B5AA89118,0,08E97D04-DB48-4B68-9231-9329FC49A4AD,2019-10-24 04:00:00,10/24/19 18:37,10/24/19 18:36,0,0.0,5B3D4939-CBAB-45F6-BC37-6BFB0EC5DA15,0,0.0
15,1574795,Level 2,Closed,Good,Full,4.0,0.0,62589BAE-7D1F-43B8-B7CD-E3D9282066C7,1B296489-CDCD-4D94-A800-AC1EF2B77302,0,ABB9C4B5-00B0-4BC8-8A7A-7D6375241CFB,2019-09-30 20:56:00,9/30/19 20:56,9/30/19 20:56,0,0.0,62589BAE-7D1F-43B8-B7CD-E3D9282066C7,0,0.0
14,1598221,Level 2,Closed,Good,Full,30.0,0.0,3AB16DFC-2205-46D8-B6D3-D9A24876EFA7,F3EA5732-84DE-4919-9322-AD75FC6EEC9A,ED9CD903-8EB8-4D2F-ACFE-F4DB274B246D,DC0628A1-8233-4133-9903-961A6FB6B540,2019-10-02 17:42:00,10/2/19 17:42,10/2/19 17:42,0,0.0,3AB16DFC-2205-46D8-B6D3-D9A24876EFA7,0,0.0
13,1587863,Level 2,Closed,Good,Full,15.0,0.0,239B2FED-7456-4824-A8F7-1AA928084952,025EEA00-CC7B-4271-B835-22FD244118BD,0,90332812-4931-4740-A0A7-8D5BC85ABC35,2019-10-01 21:47:00,10/1/19 21:47,10/1/19 21:47,0,0.0,239B2FED-7456-4824-A8F7-1AA928084952,0,0.0
12,1803447,Level 2,Closed,Good,Full,10.0,0.0,004B197F-1E95-4B97-937A-F4F38CEE2CEF,0FC27A55-4291-4F0C-9219-B52A61287690,0,E63F60D7-2363-4D70-859E-1A8CC66FDC28,2019-10-29 04:00:00,10/29/19 21:32,10/29/19 21:26,0,0.0,004B197F-1E95-4B97-937A-F4F38CEE2CEF,0,0.0
11,1749982,Level 2,Closed,Good,Full,18.0,0.0,FCD769F8-3F4D-4A3A-87D9-B8F7A98F42E3,E33D8E86-DA56-467B-9ACA-2847711D0D64,0,9B3B79D2-1264-4FE2-B845-D887F6EBF174,2019-10-24 04:00:00,10/24/19 16:14,10/24/19 16:13,0,0.0,FCD769F8-3F4D-4A3A-87D9-B8F7A98F42E3,0,0.0


In [ ]:
%insights_return(trees)

In [ ]:
%insights_return(insps)